<a href="https://colab.research.google.com/github/Kabindra02/minor-project/blob/main/sign_language.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
remote: Enumerating objects: 17270, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 17270 (delta 0), reused 0 (delta 0), pack-reused 17265 (from 1)
Receiving objects: 100% (17270/17270), 16.11 MiB | 18.24 MiB/s, done.
Resolving deltas: 100% (11861/11861), done.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:

%pwd


'/content'

In [7]:

%cd yolov5


/content/yolov5


In [8]:

# install dependencies as necessary
!pip install -qr requirements.txt  # install dependencies (ignore errors)
import torch

from IPython.display import Image, clear_output  # to display images
# from utils.google_utils import gdrive_download  # to download models/datasets

# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.9/914.9 kB 37.2 MB/s eta 0:00:00
Setup complete. Using torch 2.5.1+cu124 CPU


In [11]:
import zipfile

zip_path="/content/drive/MyDrive/ASLProject.zip"
zip_extract="/content/sign_language_dataset"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(zip_extract)

In [14]:
%cd /content/sign_language_dataset
%cd /content/sign_language_dataset/ASLProject
!ls


/content/sign_language_dataset
/content/sign_language_dataset/ASLProject
data.yaml  README.dataset.txt  README.roboflow.txt  test  train  valid


In [15]:
%cat data.yaml

train: ../train/images
val: ../valid/images
test: ../test/images

nc: 26
names: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']

roboflow:
  workspace: meredith-lo-pmqx7
  project: asl-project
  version: 17
  license: CC BY 4.0
  url: https://universe.roboflow.com/meredith-lo-pmqx7/asl-project/dataset/17

In [16]:
import yaml
with open("data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [17]:

num_classes

'26'

In [18]:
%cat /content/yolov5/models/yolov5s.yaml

# Ultralytics 🚀 AGPL-3.0 License - https://ultralytics.com/license

# Parameters
nc: 80 # number of classes
depth_multiple: 0.33 # model depth multiple
width_multiple: 0.50 # layer channel multiple
anchors:
  - [10, 13, 16, 30, 33, 23] # P3/8
  - [30, 61, 62, 45, 59, 119] # P4/16
  - [116, 90, 156, 198, 373, 326] # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [
    [-1, 1, Conv, [64, 6, 2, 2]], # 0-P1/2
    [-1, 1, Conv, [128, 3, 2]], # 1-P2/4
    [-1, 3, C3, [128]],
    [-1, 1, Conv, [256, 3, 2]], # 3-P3/8
    [-1, 6, C3, [256]],
    [-1, 1, Conv, [512, 3, 2]], # 5-P4/16
    [-1, 9, C3, [512]],
    [-1, 1, Conv, [1024, 3, 2]], # 7-P5/32
    [-1, 3, C3, [1024]],
    [-1, 1, SPPF, [1024, 5]], # 9
  ]

# YOLOv5 v6.0 head
head: [
    [-1, 1, Conv, [512, 1, 1]],
    [-1, 1, nn.Upsample, [None, 2, "nearest"]],
    [[-1, 6], 1, Concat, [1]], # cat backbone P4
    [-1, 3, C3, [512, False]], # 13

    [-1, 1, Conv, [256, 1, 1]],
    [-1, 1, nn.Upsample, [None, 2, 

In [19]:
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [20]:
%%writetemplate /content/yolov5/models/custom_yolov5s.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

### **Train Custom YOLOv5 Detector**

Next, we'll fire off training!

Here, we are able to pass a number of arguments:

**img:** define input image size

**batch:** determine batch size

**epochs:** define the number of training epochs. (Note: often, 3000+ are common here!)

**data:** set the path to our yaml file

**cfg:** specify our model configuration

**weights:** specify a custom path to weights. (Note: you can download weights from the Ultralytics Google Drive folder)

**name:** result names

**nosave:** only save the final checkpoint

**cache:** cache images for faster training

In [21]:
!ls /content/yolov5/models/  # Check if 'custom_yolov5s.yaml' exists


common.py	     hub	  tf.py		yolov5m.yaml  yolov5x.yaml
custom_yolov5s.yaml  __init__.py  yolo.py	yolov5n.yaml
experimental.py      segment	  yolov5l.yaml	yolov5s.yaml


In [22]:
!cp "/content/sign_language_dataset/ASLProject/data.yaml" /content/


In [23]:
!ls "/content/sign_language_dataset/ASLProject/"

data.yaml  README.dataset.txt  README.roboflow.txt  test  train  valid


In [30]:
!ls -l "/content/sign_language_dataset/ASLProject/valid"
!ls -la "/content/sign_language_dataset/ASLProject/valid"


total 304
drwxr-xr-x 2 root root 147456 Feb  1 11:25 images
drwxr-xr-x 2 root root 155648 Feb  1 11:25 labels
total 312
drwxr-xr-x 4 root root   4096 Feb  1 11:25 .
drwxr-xr-x 5 root root   4096 Feb  1 11:25 ..
drwxr-xr-x 2 root root 147456 Feb  1 11:25 images
drwxr-xr-x 2 root root 155648 Feb  1 11:25 labels


In [35]:
!ls -l "/content/sign_language_dataset/ASLProject/train"
!ls -la "/content/sign_language_dataset/ASLProject/valid"

total 2116
drwxr-xr-x 2 root root 1077248 Feb  1 11:26 images
drwxr-xr-x 2 root root 1081344 Feb  1 11:26 labels
total 312
drwxr-xr-x 4 root root   4096 Feb  1 11:25 .
drwxr-xr-x 5 root root   4096 Feb  1 11:25 ..
drwxr-xr-x 2 root root 147456 Feb  1 11:25 images
drwxr-xr-x 2 root root 155648 Feb  1 11:25 labels


In [39]:
import shutil

#Define the source and destination paths
source_path = '/content/sign_language_dataset/ASLProject/train'
destination_path = '/content/'

#Move the directory
shutil.move(source_path, destination_path)

print(f"Directory moved from {source_path} to {destination_path}")

Directory moved from /content/sign_language_dataset/ASLProject/train to /content/


In [41]:
!ls /content/  # Check if 'data.yaml' is in the right place

data.yaml  drive  sample_data  sign_language_dataset  train  valid  yolov5


In [48]:
# train yolov5s on custom data for 100 epochs
# time its performance
%%time
%cd /content/yolov5/
!python train.py --img 512 --batch 8 --epochs 50 --data '../data.yaml' --cfg ./models/custom_yolov5s.yaml --weights 'yolov5s.pt' --name yolov5s_results --patience 10 --cache

/content/yolov5
wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-02-01 11:57:36.910757: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738411056.935884   10633 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738411056.943274   10633 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: Currently logged in as: kabindra (kabindra-acem) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
train: weights=yolov5s.pt, cfg=./models/custom_yolov5s.yaml, data=../data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=50, batch_s